<a href="https://colab.research.google.com/github/Dhanasree-Rajamani/Deep_Learning_Assignments/blob/main/Assignment_2c/Dhanasree_Search_Sort_Tensor_2c_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Performing Binary search, Linear search and Insertion sort using tensorflow

Import statements

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Binary Search

Added comments below to explain how binary search is performed on an array

In [4]:
class BinarySearch():
    def __init__(self, array, x):
        self.array = tf.constant(array)
        self.x = tf.constant(x)
        self.loop = tf.while_loop(self.cond, self.body, [-1,False,0,len(array),-1],
                        back_prop=False)

    def run(self):
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            return sess.run(self.loop)

    # setting condition for binary search 
    # as left < right
    # and not is_found
    def cond(self, x, is_found, left, right, mid):
        return tf.logical_and(tf.less_equal(left, right), tf.logical_not(is_found))

    def body(self, x, is_found, left, right, mid):
        
        #calculating mid = (left + right)/2
        mid = tf.cast(tf.divide(tf.add(left, right), 2), tf.int32)
        return tf.cond(tf.equal(self.array[mid], self.x),
                    
                    #if array[mid] equals x set is_found as true and return mid
                    lambda: (self.array[mid], True, left, right, mid),
                    
                    #else check if array[mid] less than x, set left as mid+1
                    #else if array[mid] greater than x, then set right as mid-1
                    #and continue the loop
                    lambda: tf.cond(tf.less(self.array[mid], self.x),
                                lambda: (-1, False, tf.add(mid, 1), right, mid),
                                lambda: (-1, False, left, tf.subtract(mid, 1), mid)))

if __name__ == '__main__':
    #sort array for binary search
    array = sorted([1, 8, 7, 4, 3, 2, 6, 5])
    x = 3
    search = BinarySearch(array, x)
    number, is_found, _, _, index = search.run()

    print('Array :', array)
    print('Number to search :', x)
    if is_found:
        print(number, ' is at index ', index)
    else:
        print('Item not found.')

Array : [1, 2, 3, 4, 5, 6, 7, 8]
Number to search : 3
3  is at index  2


Linear Search

Added comments below to explain how linear search is performed on a given array

In [5]:
class LinearSearch():
    def __init__(self, array, x):
        self.x = tf.constant(x)
        self.array = tf.constant(array)
        self.length = len(array)
        #while loop which performs the linear search
        self.graph = tf.while_loop(self.cond, self.body, [0, self.x, False],
                            back_prop=False)
        
    #creating session where we run the entire loop
    def run(self):
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            return sess.run(self.graph)

    # condition to check:
    # i < length and
    # not is_found 
    def cond(self, i, _, is_found):
        return tf.logical_and(tf.less(i, self.length), tf.logical_not(is_found))

    
    # if current item equals num to search for
    # array[i] : current item, x : number to search
    # then set is_found as true
    # else increment i,  
    def body(self, i, _, is_found):
        return tf.cond(tf.equal(self.array[i], self.x),
                    lambda: (i, self.array[i], True),
                    lambda: (tf.add(i, 1), 0, False))


if __name__ == '__main__':
    #input array and value to search for
    array, x = [1, 7, 3, 8, 9, 6], 8
    search = LinearSearch(array, x)
    index, number, is_found = ret = search.run()
    print('Array :', array)
    print('Number to search :', x)
    if is_found:
        print(number, ' is at index ', index)
    else:
        print('Item not found.')

Array : [1, 7, 3, 8, 9, 6]
Number to search : 8
8  is at index  3


Insertion Sort

Added comments below to explain how insertion sort is performed on a given array

In [6]:
class InsertionSort():
    def __init__(self, array):
        self.i = tf.constant(1)
        self.j = tf.constant(len(array)-1)
        self.array = tf.Variable(array, trainable=False)
        self.length = len(array)

        # condition for while loop : i-1 < length-1
        cond = lambda i, j, _: tf.less(i-1, self.length-1)
        self.graph = tf.while_loop(cond, self.outer_loop, loop_vars=[self.i, self.j, self.array],
                shape_invariants=[self.i.get_shape(), self.j.get_shape(), tf.TensorShape(self.length)],
                parallel_iterations=1,
                back_prop=False)

    def run(self):
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            return sess.run(self.graph)

    # body of while loop
    def outer_loop(self, i, j, _):
        j = i
        # check if array has previous elements
        # and if yes, check if the previous element is greater than the current element  
        cond = lambda i, j, array: tf.logical_and(tf.greater(j,0), tf.greater(array[j-1], array[j]))

        loop = tf.while_loop(cond, self.inner_loop, loop_vars=[i, j, self.array],
                    shape_invariants=[i.get_shape(), j.get_shape(), tf.TensorShape(self.length)],
                    parallel_iterations=1,
                    back_prop=False)
        return tf.add(i, 1), loop[1], loop[2]


    def inner_loop(self, i, j, _):
        # decrement j
        # swap current element with previous element
        return i, tf.subtract(j, 1), tf.scatter_nd_update(self.array, [[j-1],[j]], [self.array[j],self.array[j-1]])

with tf.Session() as sess:
    x = np.array([1,7,3,4,8,10])
    print(x)
    print("sorted array is :")
    print(InsertionSort(x).run()[2])

[ 1  7  3  4  8 10]
sorted array is :
[ 1  3  4  7  8 10]
